In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
# test_df = pd.read_csv("/home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/_test.csv")
# test_condition = (test_df['ty'] > -60) & (test_df['tx'] > -130) & (test_df['tx'] < 20) \
#     | (test_df['ty'] > -95) & (test_df['ty'] < -50) & (test_df['tx'] > -120) & (test_df['tx'] < -40)
# test_df = test_df[test_condition]

# train_df = pd.read_csv("/home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/_train.csv")
# train_condition = (train_df['ty'] > -60) & (train_df['tx'] > -130) & (train_df['tx'] < 20) \
#     | (train_df['ty'] > -95) & (train_df['ty'] < -50) & (train_df['tx'] > -120) & (train_df['tx'] < -40)
# train_condition = ~train_condition
# train_df = train_df[train_condition]

# train_df.to_csv("/home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/train.csv", index=False)
# test_df.to_csv("/home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/test.csv", index=False)
# test_df.to_csv("/home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2/val.csv", index=False)

In [4]:
# plt.figure(figsize=(10, 6))
# plt.scatter(train_df['tx'].to_numpy(), train_df['ty'].to_numpy(), marker='o', c='b')
# plt.scatter(test_df['tx'].to_numpy(), test_df['ty'].to_numpy(), marker='o', c='r')
# plt.xlabel('tx')
# plt.ylabel('ty')
# plt.title('Trajectory')
# plt.axis('equal')
# plt.legend(['train', 'test'])
# plt.show()

In [5]:
import pprint
import sys
from datetime import datetime
from pathlib import Path
from typing import Dict, Literal

import torch
import hydra
import wandb
from hydra.utils import instantiate
from loguru import logger
from omegaconf import DictConfig, OmegaConf
from opr.datasets.dataloader_factory import make_dataloaders
from opr.trainers.place_recognition import MultimodalPlaceRecognitionTrainer
from opr.utils import set_seed
from torch.utils.data import DataLoader
from opr.datasets.itlp import ITLPCampus

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.24 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../../configs"):
    cfg = compose(config_name="finetune_itlp_multimodal")

print(OmegaConf.to_yaml(cfg))

In [7]:
DATASET_ROOT = "/home/docker_opr/Datasets/OpenPlaceRecognition/itlp_campus_outdoor_part2"

## Init wandb

In [8]:
if not cfg.debug and not cfg.wandb.disabled:
    config_dict = OmegaConf.to_container(cfg, resolve=True, throw_on_missing=True)
    wandb.init(
        name=cfg.exp_name,
        project=cfg.wandb.project,
        settings=wandb.Settings(start_method="thread"),
        config=config_dict,
    )
    run_name = wandb.run.name
else:
    run_name = "debug"

checkpoints_dir = (
    Path(cfg.checkpoints_dir) / f"{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}_{run_name}"
)
if not checkpoints_dir.exists():
    checkpoints_dir.mkdir(parents=True)

set_seed(seed=cfg.seed, make_deterministic=False)  # we cannot use determenistic operators here :(
print(f"=> Seed: {cfg.seed}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: melekhin-aa-work. Use `wandb login --relogin` to force relogin


=> Seed: 3121999


## Init dataloaders

In [9]:
train_dataset = ITLPCampus(
    dataset_root=DATASET_ROOT,
    subset="train",
    csv_file="train.csv",
    sensors=["front_cam", "back_cam", "lidar"],
)
val_dataset = ITLPCampus(
    dataset_root=DATASET_ROOT,
    subset="val",
    csv_file="val.csv",
    sensors=["front_cam", "back_cam", "lidar"],
)
test_dataset = ITLPCampus(
    dataset_root=DATASET_ROOT,
    subset="test",
    csv_file="test.csv",
    sensors=["front_cam", "back_cam", "lidar"],
)
# test_dataset.dataset_df = test_dataset.dataset_df[test_dataset.dataset_df["track"].isin(["05_2023-08-15-day", "07_2023-10-04-day"])].reset_index(drop=True)

train_sampler = instantiate(cfg.sampler, dataset=train_dataset)
val_sampler = instantiate(cfg.sampler, dataset=val_dataset)

dataloaders = {}
dataloaders["train"] = DataLoader(
    train_dataset,
    batch_sampler=train_sampler,
    collate_fn=train_dataset.collate_fn,
    num_workers=cfg.num_workers,
    pin_memory=True,
)
dataloaders["val"] = DataLoader(
    val_dataset,
    batch_sampler=val_sampler,
    collate_fn=val_dataset.collate_fn,
    num_workers=cfg.num_workers,
    pin_memory=True,
)
dataloaders["test"] = DataLoader(
    test_dataset,
    batch_size=cfg.sampler.batch_size_limit,
    collate_fn=test_dataset.collate_fn,
    num_workers=cfg.num_workers,
    pin_memory=True,
)

In [10]:
len(dataloaders["test"].dataset.dataset_df)

610

## Init loss

In [11]:
loss_fn = instantiate(cfg.loss)

## Init model

In [12]:
model = instantiate(cfg.model)

# load pretrained NCLT checkpoint
ckpt = torch.load("/home/docker_opr/OpenPlaceRecognition/weights/place_recognition/multi-image_lidar_late-fusion_nclt.pth")
model.load_state_dict(ckpt)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


<All keys matched successfully>

## Init optimizer and scheduler

In [13]:
optimizer = instantiate(cfg.optimizer, params=model.parameters())
scheduler = instantiate(cfg.scheduler, optimizer=optimizer)

In [14]:
trainer = MultimodalPlaceRecognitionTrainer(
    modalities_weights=cfg.modalities_weights,
    checkpoints_dir=checkpoints_dir,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    scheduler=scheduler,
    batch_expansion_threshold=cfg.batch_expansion_threshold,
    wandb_log=(not cfg.debug and not cfg.wandb.disabled),
    device=cfg.device,
)

In [ ]:
trainer.train(epochs=cfg.epochs, train_dataloader=dataloaders["train"], val_dataloader=dataloaders["val"], test_dataloader=dataloaders["test"])

In [16]:
best_ckpt = torch.load(str(checkpoints_dir / "best.pth"))
trainer.model.load_state_dict(best_ckpt["model_state_dict"])

<All keys matched successfully>

In [17]:
trainer.test(dataloaders["test"])

2024-12-27 11:37:17.158 | INFO     | opr.trainers.place_recognition.unimodal:test:172 - => Test stage:
2024-12-27 11:37:28.833 | DEBUG    | opr.trainers.place_recognition.unimodal:test:194 - Test embeddings: (610, 512)
2024-12-27 11:37:28.994 | INFO     | opr.trainers.place_recognition.unimodal:test:235 - Test time: 00:11
2024-12-27 11:37:28.995 | INFO     | opr.trainers.place_recognition.unimodal:test:236 - Mean Recall@N:
[0.99509334 0.99782782 1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.        ]
2024-12-27 11:37:28.996 | INFO     | opr.trainers.place_recognition.unimodal:test:237 - Mean Recall@1% = 0.9978278227620333
2024-12-27 11:37:28.996 | INFO     | opr.trainers.place_recognition.unimodal:test:238 - Mean top-1 distance = 1.1823934802782412
